In [ ]:
%pylab
import sys; sys.path.append('..')
import opencl as cl

In [ ]:
# module documentation
help(cl.get_compute_device)
help(cl.ComputeDevice)
help(cl.Program)
help(cl.Kernel.__call__)
help(cl.Image)
help(cl.Buffer)

## hints
- Documentation for OpenCl (use 1.2) can be donloaded from https://www.khronos.org/registry/OpenCL/. The overview over available functions is downloadable from https://www.khronos.org/registry/cl/sdk/1.2/docs/OpenCL-1.2-refcard.pdf.
- For writing good code, you need to understand some important concepts of OpenCl and GPU programming. These are at least: 
    - Parallel execution in work groups, 
    - caching, texture memory (`cl.Image` objects) and `constant` buffers, 
    - execution queue,
    - performance bottlenecks (memory bandwith vs. computation)
    - ... in addition to more basic computation concepts (e.g. data types int/float, ...).
- On Windows, long kernel executions (Kernel.\_\_call\_\_ has runtime over 2 s) can lead to arbitrary program crashes due to a "bug" in the operating system. Solution: split computation into several calls, e.g. by tiling an image.

In the following cell, a very simple example of an OpenCl kernel execution is shown.

In [ ]:
# a device must be initialized before any computations can be done (should only be executed once)
cl_device = cl.get_compute_device()
cl.CHECK_KERNEL_ARGS = True # enable debugging feature of the opencl module

In [13]:
# first step: write OpenCl kernels (the language is based on C)
cl_source = '''
constant sampler_t nearest_sampler = CLK_NORMALIZED_COORDS_FALSE | CLK_ADDRESS_CLAMP | CLK_FILTER_NEAREST;
// change the sample e.g. to use linear interpolation (which has +0.5f coordinate offset compared to nearest)

kernel void compute(read_only image2d_t image_in, global float* image_out, float addition) {
    const int2 pos = (int2)(get_global_id(0), get_global_id(1));
    const int2 shape = get_image_dim(image_in);
    if (all(pos < shape)) {
        float value = read_imagef(image_in, nearest_sampler, pos).x;
        
        if (fabs(value) > 1.0f){value = sign(value)*(2.0f+log(fabs(value)));}
        else{value = exp(value);}
        
        value = exp(-fabs(value));
        
        if (value < 0.f){value = 0.f;}
        
        value += addition;
        
        image_out[shape.x*pos.y+pos.x] = value;
    }
}
'''

# second step: compile kernels
program = cl.Program(cl_device, cl_source)
print(program)

# third step: allocate device memory
image = random.randn(221, 301).astype('f4') # input data on CPU memory
image_cl = cl.Image(cl_device, arr=image) # allocate GPU memory and copy data (default arg usage)
result_cl = cl.Buffer(cl_device, size=image.nbytes) # allocate GPU memory for result
print(image_cl, result_cl, sep='\n')

# fourth step: execute kernel on device
program.compute(image.shape, (16, 16), image_cl, result_cl, float32(1.0))

# fifth step: retrieve result (device => Python)
result = zeros_like(image)
result_cl.copy_to(result)
print('means:', image.mean(), result.mean())

===== OpenCl Program # 60215712 with 1 kernels =====
compute(image2d_t image_in, float* image_out, float addition)
===== end kernels =====
Image with shape (221, 301) and image format (INTENSITY, FLOAT) (id 60372800)
Buffer with size 266084 (id c_ulong(44216096))
means: -0.0042030094 1.2831137


In [12]:
print(image[0, :5], '\n', result[0, :5])

[ 0.5117388   0.05801357  0.8154782   1.1815238  -1.1697786 ] 
 [1.1885883 1.3465495 1.1043237 1.114543  1.1156931]


# development: finding errors

In [ ]:
# errors in the kernel code are shown this way
wrong_cl_source = cl_source.replace('value += addition;', 'value += addition')
cl.Program(cl_device, wrong_cl_source)

In [ ]:
# wrong argument variant 1
program.compute(image.shape, (16, 16), image, result_cl, float32(1.0))

In [ ]:
# wrong argument variant 2 (only raises an Exception when cl.CHECK_KERNEL_ARGS is enabled)
program.compute(image.shape, (16, 16), image_cl, result_cl, int32(1.0))